# PostgreSQL Summary Stats and Window Functions
Here you can access the `summer_medals` table used in the course. To access the table, you will need to specify the `medals` schema in your queries (e.g., `medals.summer_medals`).


## Take Notes

Add notes about the concepts you've learned and SQL cells with queries you want to keep.

_Add your notes here_

In [ ]:
-- Add your own queries here
SELECT *
FROM medals.summer_medals
LIMIT 5

## Explore Datasets
Use the `summer_medals` table to explore the data and practice your skills!
- Select the `athlete`, `event`, and `year` from the `summer_medals` table.
    - Add another column, `previous_winner`, which contains the previous winner of the same event.
    - Filter your results for gold medalists.
- Return the `year`, total number of medalists per year, and running total number of medalists in the history of the Summer Olympics.
    - Order your results by year in ascending order.
- Return the `country`, `year`, and the number of gold medals earned.
   - Limit your results to the years 2004, 2008, and 2012.
   - Each country should have a subtotal of all gold medals earned across the three years.

In [1]:
SELECT
  *,
  -- Assign numbers to each row
  ROW_NUMBER() OVER() AS Row_N
FROM medals.summer_medals
ORDER BY Row_N ASC;

In [3]:
SELECT
  Year,
  -- Assign numbers to each year
  ROW_NUMBER() OVER() AS Row_N
FROM (
  SELECT DISTINCT YEAR
  FROM medals.summer_medals
  ORDER BY Year ASC
) AS Years
ORDER BY Year ASC;


,year,row_n
0,1896,1
1,1900,2
2,1904,3
3,1908,4
4,1912,5
5,1920,6
6,1924,7
7,1928,8
8,1932,9
9,1936,10


In [4]:
SELECT
  Year,
  -- Assign the lowest numbers to the most recent years
  ROW_NUMBER() OVER (ORDER BY Year DESC) AS Row_N
FROM (
  SELECT DISTINCT Year
  FROM medals.summer_medals
) AS Years
ORDER BY Year;

,year,row_n
0,1896,27
1,1900,26
2,1904,25
3,1908,24
4,1912,23
5,1920,22
6,1924,21
7,1928,20
8,1932,19
9,1936,18


In [5]:
SELECT
  -- Count the number of medals each athlete has earned
  athlete,
  COUNT(medal) AS Medals
FROM medals.summer_medals
GROUP BY athlete
ORDER BY Medals DESC;

,athlete,medals
0,PHELPS Michael,22
1,LATYNINA Larisa,18
2,ANDRIANOV Nikolay,15
3,MANGIAROTTI Edoardo,13
4,ONO Takashi,13
...,...,...
22757,TOMAN Ladislav,1
22758,OUTTERIDGE Nathan,1
22759,LISKOVA Hana,1
22760,POLENOVA Elena,1


In [6]:
WITH Athlete_Medals AS (
  SELECT
    -- Count the number of medals each athlete has earned
    athlete,
    COUNT(*) AS Medals
  FROM medals.summer_medals
  GROUP BY athlete)

SELECT
  -- Number each athlete by how many medals they've earned
  athlete,
  ROW_NUMBER() OVER (ORDER BY Medals DESC) AS Row_N
FROM Athlete_Medals
ORDER BY Medals DESC;

,athlete,row_n
0,PHELPS Michael,1
1,LATYNINA Larisa,2
2,ANDRIANOV Nikolay,3
3,MANGIAROTTI Edoardo,4
4,ONO Takashi,5
...,...,...
22757,TOMAN Ladislav,22758
22758,OUTTERIDGE Nathan,22759
22759,LISKOVA Hana,22760
22760,POLENOVA Elena,22761


In [7]:
WITH Weightlifting_Gold AS (
  SELECT
    -- Return each year's champions' countries
    Year,
    Country AS champion
  FROM medals.summer_medals
  WHERE
    Discipline = 'Weightlifting' AND
    Event = '69KG' AND
    Gender = 'Men' AND
    Medal = 'Gold')

SELECT
  Year, Champion,
  -- Fetch the previous year's champion
  LAG(Champion) OVER
    (ORDER BY YEAR ASC) AS Last_Champion
FROM Weightlifting_Gold
ORDER BY Year ASC;

,year,champion,last_champion
0,2000,BUL,None
1,2004,CHN,BUL
2,2008,CHN,CHN
3,2012,CHN,CHN


In [8]:
WITH Tennis_Gold AS (
  SELECT DISTINCT
    Gender, Year, Country
  FROM medals.summer_medals
  WHERE
    Year >= 2000 AND
    Event = 'Javelin Throw' AND
    Medal = 'Gold')

SELECT
  Gender, Year,
  Country AS Champion,
  -- Fetch the previous year's champion by gender
  LAG(Country) OVER (PARTITION BY Gender
            ORDER BY YEAR ASC) AS Last_Champion
FROM Tennis_Gold
ORDER BY Gender ASC, Year ASC;

,gender,year,champion,last_champion
0,Men,2000,CZE,None
1,Men,2004,NOR,CZE
2,Men,2008,NOR,NOR
3,Men,2012,TTO,NOR
4,Women,2000,NOR,None
5,Women,2004,CUB,NOR
6,Women,2008,CZE,CUB
7,Women,2012,CZE,CZE


In [9]:
WITH Athletics_Gold AS (
  SELECT DISTINCT
    Gender, Year, Event, Country
  FROM medals.summer_medals
  WHERE
    Year >= 2000 AND
    Discipline = 'Athletics' AND
    Event IN ('100M', '10000M') AND
    Medal = 'Gold')

SELECT
  Gender, Year, Event,
  Country AS Champion,
  -- Fetch the previous year's champion by gender and event
  LAG(Country) OVER (PARTITION BY (Gender, event)
            ORDER BY Year ASC) AS Last_Champion
FROM Athletics_Gold
ORDER BY Event ASC, Gender ASC, Year ASC;

,gender,year,event,champion,last_champion
0,Men,2000,10000M,ETH,None
1,Men,2004,10000M,ETH,ETH
2,Men,2008,10000M,ETH,ETH
3,Men,2012,10000M,GBR,ETH
4,Women,2000,10000M,ETH,None
5,Women,2004,10000M,CHN,ETH
6,Women,2008,10000M,ETH,CHN
7,Women,2012,10000M,ETH,ETH
8,Men,2000,100M,USA,None
9,Men,2004,100M,USA,USA


# fetching - Relative (lead and Lag) and Absolute (first_value and last_value)

In [1]:
WITH Discus_Medalists AS (
  SELECT DISTINCT
    Year,
    Athlete
  FROM medals.summer_medals
  WHERE Medal = 'Gold'
    AND Event = 'Discus Throw'
    AND Gender = 'Women'
    AND Year >= 2000)

SELECT
  -- For each year, fetch the current and future medalists
  Year,
  Athlete,
  LEAD(Athlete,3) OVER (ORDER BY Year ASC) AS Future_Champion
FROM Discus_Medalists
ORDER BY Year ASC;

,year,athlete,future_champion
0,2000,ZVEREVA Ellina,PERKOVIC Sandra
1,2004,SADOVA Natalya,None
2,2008,BROWN TRAFTON Stephanie,None
3,2012,PERKOVIC Sandra,None


In [2]:
WITH All_Male_Medalists AS (
  SELECT DISTINCT
    Athlete
  FROM medals.summer_medals
  WHERE Medal = 'Gold'
    AND Gender = 'Men')

SELECT
  -- Fetch all athletes and the first athlete alphabetically
  Athlete,
  FIRST_VALUE(Athlete) OVER (
    ORDER BY Athlete ASC
  ) AS First_Athlete
FROM All_Male_Medalists;

,athlete,first_athlete
0,AABYE Edgar,AABYE Edgar
1,AALTONEN Paavo Johannes,AABYE Edgar
2,AAS Thomas Valentin,AABYE Edgar
3,ABALMASAU Aliaksei,AABYE Edgar
4,ABALO Luc,AABYE Edgar
...,...,...
6240,ZÜRNER Albert,AABYE Edgar
6241,ZUTTER Louis,AABYE Edgar
6242,ZVIADAURI Zurab,AABYE Edgar
6243,ZWERVER Ronald,AABYE Edgar


In [3]:
WITH Hosts AS (
  SELECT DISTINCT Year, City
    FROM medals.summer_medals)

SELECT
  Year,
  City,
  -- Get the last city in which the Olympic games were held
  LAST_VALUE(City) OVER (
   ORDER BY Year ASC
   RANGE BETWEEN
     UNBOUNDED PRECEDING AND
     UNBOUNDED FOLLOWING
  ) AS Last_City
FROM Hosts
ORDER BY Year ASC;

,year,city,last_city
0,1896,Athens,London
1,1900,Paris,London
2,1904,St Louis,London
3,1908,London,London
4,1912,Stockholm,London
5,1920,Antwerp,London
6,1924,Paris,London
7,1928,Amsterdam,London
8,1932,Los Angeles,London
9,1936,Berlin,London
